# Erstellen eines einfachen Chatbots from Scratch in Python (mit NLTK)

Die Geschichte der Chatbots geht auf das Jahr 1966 zurück, als Weizenbaum ein Computerprogramm namens ELIZA erfand. Es imitierte die Sprache eines Psychotherapeuten aus nur 200 Zeilen Code. Siehe: [Eliza](http://psych.fullerton.edu/mbirnbaum/psych101/Eliza.htm?utm_source=ubisend.com&utm_medium=blog-link&utm_campaign=ubisend). 

Ähnlich wie Weizenbaum erstellen wir einen sehr einfachen Chatbot, der jedoch die NLTK-Bibliothek von Python verwendet. Einen sehr einfachen Bot mit kaum kognitiven Fähigkeiten, aber ein kleiner Einstieg in NLP einzusteigen und um Chatbots kennenzulernen....

## Module importieren

In [1]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import text 
import warnings
warnings.filterwarnings('ignore')

## NLTK importieren
NLTK(Natural Language Toolkit) ist eine führende Plattform für die Erstellung von Python-Programmen zur Natural Language Programming.. Es bietet benutzerfreundliche Schnittstellen zu über 50 Korpora- und lexikalischen Ressourcen wie WordNet sowie eine Reihe von Textverarbeitungsbibliotheken für Klassifizierung, Tokenisierung, Stemming, Tagging, Parsing und semantisches Denken, auch Wrapper für industrietaugliche NLP-Bibliotheken.

[Natural Language Processing with Python](http://www.nltk.org/book/) bietet eine praktische Einführung in die Programmierung der Sprachverarbeitung.

In [13]:
import nltk
from nltk.stem.snowball import GermanStemmer #from nltk.stem import german stemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction import text # um stopwortliste zu erweitern
nltk.download('popular', quiet=True) # for downloading packages
#nltk.download('punkt') # first-time use only
#nltk.download('wordnet') # first-time use only

True

## Eigenen Text einlesen, z.B.:

Die Reden von Christian Lindner von 2017-2019

In [3]:
f=open('./data/lindner-talks_17-19.txt','r',errors = 'ignore', encoding='utf-8')
raw = f.read()
raw = raw.lower()# converts to lowercase

Vorverarbeitung des Textdatensatzes in Schritten:

* Umwandlung des gesamten Textes in **Großbuchstaben** oder **Kleinbuchstaben**, so dass der Algorithmus nicht in verschiedenen Fällen die gleichen Wörter als unterschiedlich behandelt.

* * **Tokenisierung**: Tokenisierung ist nur der Begriff, der verwendet wird, um den Prozess der Umwandlung der normalen Textzeichenketten in eine Liste von Token zu beschreiben.

Das NLTK-Datenpaket enthält einen pretrained Punkt-Tokenizer für Englisch

* Entfernen von **Rauschen**, d.h. alles, was nicht in einer Standardzahl oder einem Standardbrief enthalten ist.

* Entfernen der **Stop-Wörter**. Manchmal werden einige sehr gebräuchliche Wörter, vollständig aus dem Vokabular ausgeschlossen. Diese Wörter werden als Stoppwörter bezeichnet.

* * **Stemming**: Stemming ist der Prozess der Reduzierung von gebeugten (oder manchmal abgeleiteten) Wörtern auf ihre Stamm-, Basis- oder Wurzelform. (wir benutzen den nltk_german-stemmer) 

* * **Lemmatisierung**: Eine leichte Variante des Stemmens ist die Lemmatisierung. Der Hauptunterschied zwischen diesen besteht darin, dass das Stemmen oft nicht existierende Wörter hervorbringt, während Lemmata tatsächliche Wörter sind. 
Ein Beispiel für Lemmatisierung ist, dass das Wort "better" und "good" in derselben Lemma stehen, so dass sie als gleich angesehen werden.

## Tokenisation

In [15]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

## Preprocessing

Wir werden nun unseren individuellen Tokenizer als eine `Funktion namens my_tokenizer definieren`, die die Token als Eingabe übernimmt und normalisierte Token zurückgibt.

In [16]:
#deutsche stopwörter in die stopwords-datenbank hinzufügen

#1. stopwörter finden:

stop_words=set(stopwords.words("german"))
#print(stop_words)

#2. stopwörter hinzufügen
my_stop_words = text.ENGLISH_STOP_WORDS.union(["diesen", "es", "für", "hin", "sehr", "deinem", "ihrem", "ob", "anderes", "eurer", "jede", "dass", "der", "ihn", "ist", "zu", "bis", "einen", "hinter", "sind", "unseres", "würden", "euch", "deinen", "meines", "warst", "anderen", "mich", "muss", "am", "von", "wo", "sollte", "ich", "keine", "deine", "meinen", "nichts", "nicht", "diesem", "hat", "jenem", "wollen", "mir", "habe", "dort", "solchen", "seine", "die", "etwas", "dein", "könnte", "solche", "können", "keinem", "dieselben", "jenes", "auch", "einiger", "diese", "weil", "jener", "gewesen", "kein", "nun", "anderer", "welcher", "anderem", "haben", "dieses", "zur", "was", "durch", "ihnen", "mein", "weiter", "im", "wenn", "wieder", "werde", "selbst", "keines", "ihrer", "nur", "eure", "andern", "ander", "zwar", "bei", "ein", "würde", "man", "aller", "werden", "anders", "anderm", "machen", "aus", "seinem", "auf", "in", "sondern", "meinem", "sein", "waren", "während", "welchen", "desselben", "unseren", "alles", "also", "denselben", "euren", "manches", "jedes", "unter", "dasselbe", "eurem", "zwischen", "musste", "eures", "euer", "will", "da", "dann", "soll", "über", "sich", "seines", "jetzt", "dazu", "manche", "hier", "dies", "allen", "damit", "deiner", "doch", "meine", "derselben", "jenen", "einigem", "seiner", "einmal", "einer", "manchem", "demselben", "unsere", "wir", "derselbe", "welche", "jedem", "und", "einiges", "hatten", "war", "keiner", "einig", "zum", "wie", "denn", "du", "einige", "einigen", "welchem", "ihm", "des", "einem", "das", "andere", "eines", "dich", "alle", "vor", "weg", "uns", "meiner", "dessen", "jene", "sie", "derer", "ihr", "solcher", "hab", "er", "noch", "bist", "anderr", "manchen", "deines", "dieser", "mit", "als", "an", "dem", "keinen", "den", "ihres", "eine", "bin", "unserem", "solchem", "wollte", "so", "jeder", "hatte", "jeden", "wirst", "ohne", "dir", "allem", "ins", "seinen", "ihren", "oder", "viel", "welches", "solches", "unser", "wird", "nach", "um", "aber", "daß", "vom", "sonst", "ihre", "dieselbe", "kann", "gegen", "indem", "mancher"])
#print(my_stop_words)

#3. deutschen tokenizer als Funktion definieren

def my_tokenizer(doc):
   stemmer = GermanStemmer()
   return(' '.join ([stemmer.stem(t.lower()) 
                     for t in nltk.word_tokenize(doc) 
                     if t.lower() 
                     not in my_stop_words]))

## Keyword-Matching

Als nächstes definieren wir eine Funktion für eine Begrüßung durch den Bot, d.h. wenn die Eingabe eines Benutzers eine Begrüßung ist, soll der Bot eine Begrüßungsantwort zurückgeben. ELIZA verwendet ein einfaches Keyword-Matching für Begrüßungen. Wir werden hier das gleiche Konzept anwenden.

In [17]:
GREETING_INPUTS = ("hallo", "guten tag", "servus", "wie geht es ihnen", "wie geht es dir","hi",)
GREETING_RESPONSES = ["lieber parteifreund", "lieber mensch", "*nods*", "hallo", "lieber freund", "ich freue mich, dass sie sich mit mir unterhalten möchten"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

## Generierung von Antworten

### Bag of Words
Nach der ersten Vorverarbeitungsphase wandeln wir den Text in einen sinnvollen Vektor (oder ein Array) von Zahlen um. Der BoW ist eine Darstellung von Text, der das Auftreten von Wörtern in einem Dokument beschreibt. Es geht um zwei Dinge:

* Ein Vokabular bekannter Wörter.

* Ein Maß für das Vorhandensein bekannter Wörter.

Warum wird es als "Bag" mit Wörtern bezeichnet? Das liegt daran, dass alle Informationen über die Reihenfolge oder Struktur der Wörter im Dokument verworfen werden und das Modell sich nur damit beschäftigt, **ob die bekannten Wörter im Dokument vorkommen, nicht aber, wo sie im Dokument vorkommen.**

Die Intuition hinter dem Bag of Words ist, dass Dokumente ähnlich sind, wenn sie einen ähnlichen Inhalt haben. Auch können wir etwas über die Bedeutung des Dokuments allein aus seinem Inhalt lernen.

Wenn unser Wörterbuch zum Beispiel die Wörter {Learning, is, the, not, great} enthält und wir den Text "Learning is great" vektorisieren wollen, hätten wir den folgenden Vektor: (1, 1, 0, 0, 1).


### TF-IDF-Ansatz
Ein Problem mit dem Bag of Words-Ansatz ist, dass sehr häufige Wörter beginnen, im Dokument zu dominieren (z.B. größere Punktzahl), aber möglicherweise nicht so viel "Informationsgehalt" enthalten. Außerdem wird es längeren Dokumenten mehr Gewicht verleihen als kürzeren Dokumenten.

Ein Ansatz besteht darin, die Häufigkeit der Wörter danach zu skalieren, wie oft sie in allen Dokumenten vorkommen, so dass die Ergebnisse für häufige Wörter wie "die", die auch in allen Dokumenten häufig vorkommen, bestraft werden. Dieser Ansatz zur Bewertung wird als Term Frequency-Inverse Document Frequency, kurz TF-IDF, bezeichnet:

**Term Frequency: ist eine Bewertung der Häufigkeit des Wortes im aktuellen Dokument.**

```
TF = (Anzahl der Erscheinungen von Begriff t in einem Dokument)/(Anzahl der Begriffe im Dokument)
```

**Inverse Document Frequency: ist eine Bewertung, wie selten das Wort in allen Dokumenten ist.**

```
IDF = 1+log(N/n), wobei N die Anzahl der Dokumente und n die Anzahl der Dokumente ist, in denen ein Begriff t erschienen ist.
```
### Cosine Similarity

Tf-idf-Gewicht ist ein Gewicht, das häufig bei der Informationsbeschaffung und beim Text-Mining verwendet wird. Diese Gewichtung ist ein statistisches Maß, um zu beurteilen, wie wichtig ein Wort für ein Dokument in einer Sammlung oder einem Korpus ist.

```
Kosinusähnlichkeit (d1, d2) = Punktprodukt (d1, d2) / |||d1|||||| * |||d2|||||
```
wobei d1,d2 zwei Nicht-Null-Vektoren sind.

---
Um eine Antwort von unserem Bot für Eingabefragen zu generieren, wird das Konzept der Dokumentenähnlichkeit (document similarity) verwendet. Wir definieren eine Funktionsantwort, die die Äußerung des Benutzers nach einem oder mehreren bekannten Schlüsselwörtern durchsucht und eine von mehreren möglichen Antworten zurückgibt. Wenn es die Eingabe, die einem der Schlüsselwörter entspricht, nicht findet, gibt es eine Antwort zurück: "Entschuldigen Sie bitte, ich verstehe Sie nicht!"

In [7]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    #using my own stopwordlist
#    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='my_stop_words')
    TfidfVec = TfidfVectorizer(preprocessor=my_tokenizer)
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"Entschuldigen Sie bitte, ich verstehe Sie nicht!"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response



Schließlich werden wir die Zeilen eingeben, die unser Bot sagen soll, während er ein Gespräch beginnt und beendet, je nach Eingabe des Benutzers.

In [ ]:
flag=True
print("FDBOT: Mein Name ist Christian Lindner. Ich beantworte Ihre Fragen zu unseren Werten. Wenn Sie genug haben, tippen Sie Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("FDBOT: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("FDBOT: "+greeting(user_response))
            else:
                print("FDBOT: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("FDBOT: Auf Wiedersehn!")

FDBOT: Mein Name ist Christian Lindner. Ich beantworte Ihre Fragen zu unseren Werten. Wenn Sie genug haben, tippen Sie Bye!
hallo
FDBOT: ich freue mich, dass sie sich mit mir unterhalten möchten
wie stehts mit ihren werten?
FDBOT: wie muss man das dann werten?
für welche Werte stehen sie ein?
FDBOT: wie muss man das dann werten?
Für welche Werte steht die FDP?
FDBOT: wie muss man das dann werten?
Erzählen Sie mir von Ihrem Parteiprogramm
FDBOT: die
wettbewerbsfähigkeit deutschlands in der zukunft und die fairness unserer gesellschaft
werden nicht zuerst vom steuersystem sondern vom bildungssystem entschieden und
deshalb beginnt unsere politische erzählung an dieser stelle.
welche Rolle wpielt künstliche intelligenz für sie?
FDBOT: bis 2020 sollen
50 forschungseinrichtungen für künstliche intelligenz entstehen.
Wo?
FDBOT: Entschuldigen Sie bitte, ich verstehe Sie nicht!
wo entstehen diese forschungseinrichtungen?
FDBOT: bis 2020 sollen
50 forschungseinrichtungen für künstliche intellige